In [2]:
import requests


In [3]:

API_URL = "https://fr.wikipedia.org/w/api.php"
UA = "AnonEditStatBot/1.0 (mailto:ton.email@example.com)"


In [4]:

def anon_edit_percentage(page_title: str, start: str, end: str):
    """
    Calcule la % d'éditions anonymes (IP) sur la page page_title,
    entre les dates start et end (format 'YYYY-MM-DD').
    """
    session = requests.Session()
    session.headers.update({"User-Agent": UA})
    
    params = {
        "action":    "query",
        "format":    "json",
        "prop":      "revisions",
        "titles":    page_title,
        "rvstart":    f"{end}T23:59:59Z",
        "rvend":      f"{start}T00:00:00Z",
        "rvprop":    "user|flags|timestamp",
        "rvlimit":   "max",
        "continue":  ""
    }
    
    total = 0
    anon  = 0
    
    while True:
        resp = session.get(API_URL, params=params)
        resp.raise_for_status()
        data = resp.json()
        
        pages = data.get("query", {}).get("pages", {})
        for p in pages.values():
            for rev in p.get("revisions", []):
                total += 1
                if "anon" in rev:
                    anon += 1
        
        if "continue" in data:
            params.update(data["continue"])
        else:
            break
    
    if total == 0:
        return 0.0, anon, total
    pct = (anon / total) * 100
    return pct, anon, total


In [5]:

if __name__ == "__main__":
    page  = "Guerre"
    start = "2024-01-01"
    end   = "2025-01-01"
    pct, anon, tot = anon_edit_percentage(page, start, end)
    print(f"Sur « {page} » du {start} au {end} :")
    print(f" • {anon} éditions anonymes sur {tot} au total")
    print(f" • soit {pct:.1f}% d’éditions par IP/comptes non-enregistrés")


Sur « Guerre » du 2024-01-01 au 2025-01-01 :
 • 116 éditions anonymes sur 184 au total
 • soit 63.0% d’éditions par IP/comptes non-enregistrés
